# RCT replication of results

Steps:
- Download the results
- Find only the RCT results
- Filter for RCT results we trust
- Graph the survey responses

Download the results

In [8]:
import boto3
import pandas as pd

session = boto3.Session()
credentials = session.get_credentials()

dynamodb = boto3.resource("dynamodb", region_name="eu-west-2")
responses_table = dynamodb.Table("caddyResponses-prod")

all_items = []

response = responses_table.scan()
all_items.extend(response['Items'])
while 'LastEvaluatedKey' in response:
    response = responses_table.scan(ExclusiveStartKey=response['LastEvaluatedKey'])
    all_items.extend(response['Items'])

df_responses = pd.DataFrame(all_items)
df_responses['Office'] = df_responses.apply(lambda x: str(x['approverEmail']).split('@')[-1], axis=1)
df_responses.shape

(1057, 16)

In [11]:
messages_table = dynamodb.Table("caddyMessages-prod")

all_items = []

message = messages_table.scan()
all_items.extend(message['Items'])
while 'LastEvaluatedKey' in message:
    message = messages_table.scan(ExclusiveStartKey=message['LastEvaluatedKey'])
    all_items.extend(message['Items'])

df_messages = pd.DataFrame(all_items)
df_messages.shape

(1383, 8)

In [12]:
evaluation_table = dynamodb.Table("caddyEvaluation-prod")

all_items = []

evaluation = evaluation_table.scan()
all_items.extend(evaluation['Items'])
while 'LastEvaluatedKey' in evaluation:
    evaluation = evaluation_table.scan(ExclusiveStartKey=evaluation['LastEvaluatedKey'])
    all_items.extend(evaluation['Items'])

# Convert the list of items to a DataFrame
df_evaluation = pd.DataFrame(all_items)
df_evaluation.shape

(1019, 9)

In [16]:
df_evaluation

,threadId,callComplete,surveyResponse,continueConversation,controlGroupMessage,moduleOutputs,callStart,modulesUsed,surveyCompleteTimestamp
0,R43btk4cxhw,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Lbtn-GJ1OIo,NaN,[{'How confident did you feel in providing adv...,NaN,NaN,NaN,NaN,NaN,NaN
2,ogTQz8PHcg8,True,[{'How confident did you feel in providing adv...,True,None,"{""randomisation"": [""continue_interaction"", ""tr...",04-07-2024 11:09,"{'module_arguments': {'split': '0', 'control_g...",04-07-2024 11:11:04
3,3kPiyGnCrRo,True,[{'How confident did you feel in providing adv...,True,None,"{""randomisation"": [""continue_interaction"", ""tr...",01-07-2024 11:10,"{'module_arguments': {'split': '0', 'control_g...",01-07-2024 11:32:19
4,yJ1bQ8t0i0I,NaN,[{'How easy or difficult was it to find the in...,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
1014,2nbW3PLBD1k,False,NaN,True,None,"{""randomisation"": [""continue_interaction"", ""tr...",NaN,"{'module_arguments': {'split': '0', 'control_g...",NaN
1015,Krpn864SkVg,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1016,nv49Kd-d1BM,True,[{'How confident did you feel in providing adv...,True,None,"{""randomisation"": [""continue_interaction"", ""tr...",06-06-2024 15:14,"{'module_arguments': {'split': '0.5', 'control...",06-06-2024 15:39:46
1017,nYso56QzSas,True,[{'How confident did you feel in providing adv...,True,None,"{""randomisation"": [""continue_interaction"", ""tr...",08-08-2024 13:09,"{'module_arguments': {'split': '0', 'control_g...",14-08-2024 10:57:27


In [13]:
df_responses.sort_values(by='approvalTimestamp')

,approvalTimestamp,approverEmail,approverReceivedTimestamp,llmResponseTimestamp,userThankedTimestamp,approved,llmPromptTimestamp,messageId,llmAnswer,supervisorMessage,llmResponseJSon,responseId,llmPrompt,threadId,userResponseTimestamp,Office
672,2024-04-15 14:37:06.804700+00:00,ayesha.qudsia@casort.org,2024-04-15 14:36:30.993319,2024-04-15 14:36:28.180783,2024-04-15 14:36:30.998646,True,2024-04-15 14:35:51.232355,HwwXi0Utkb0.oP0u2kf3Ieo,"Based on the information provided, here is my ...",None,"{""cardsV2"": [{""cardId"": ""aiResponseCard"", ""car...",513feb01-7077-4ec6-9573-c465e02726af,"I have received a section 21 notice today, w...",HwwXi0Utkb0,2024-04-15 14:37:12.254120,casort.org
684,2024-04-16 08:19:46.102891+00:00,gary.collins@casort.org,2024-04-16 08:08:57.287468,2024-04-16 08:08:54.481292,2024-04-16 08:08:57.294067,True,2024-04-16 08:08:18.364832,93ddIp5cXFU.857qfA-BnwU,"Based on the information provided, here are th...",None,"{""cardsV2"": [{""cardId"": ""aiResponseCard"", ""car...",76a3a82e-095c-4e81-b8d6-17139db6ad60,list energy suppliers offer support for energ...,93ddIp5cXFU,2024-04-16 08:19:52.172572,casort.org
421,2024-04-16 08:53:43.943781+00:00,gary.collins@casort.org,2024-04-16 08:53:12.731285,2024-04-16 08:53:10.103406,2024-04-16 08:53:12.736832,True,2024-04-16 08:52:42.947325,OIcB2_ZOfhM.j90RGsepmkA,"Based on the information provided, here is my ...",None,"{""cardsV2"": [{""cardId"": ""aiResponseCard"", ""car...",513feb01-7077-4ec6-9573-c465e02726af,Client was given indefinite leave to remain b...,OIcB2_ZOfhM,2024-04-16 08:53:49.956472,casort.org
1043,2024-04-16 09:50:18.190986+00:00,gary.collins@casort.org,2024-04-16 09:48:58.212703,2024-04-16 09:48:55.803405,2024-04-16 09:48:58.217877,True,2024-04-16 09:44:43.914402,bbWQ_mxpdCk.Z3PGYpwYHpM,"Based on the information provided, your Univer...",None,"{""cardsV2"": [{""cardId"": ""aiResponseCard"", ""car...",e3288c10-c9f7-4f4d-858b-0889894e060e,My UC payments are fluctuating month to month...,bbWQ_mxpdCk,2024-04-16 09:50:23.367931,casort.org
524,2024-04-16 10:54:26.442263+00:00,ayesha.qudsia@casort.org,2024-04-16 10:51:47.174622,2024-04-16 10:51:44.181136,2024-04-16 10:51:47.180733,True,2024-04-16 10:51:13.853166,oGTmcy1pezQ.tdRFpBSjpfw,"Based on the information provided, <b>you may ...",None,"{""cardsV2"": [{""cardId"": ""aiResponseCard"", ""car...",76a3a82e-095c-4e81-b8d6-17139db6ad60,"I am over state pension age, i am not eligibl...",oGTmcy1pezQ,2024-04-16 10:54:32.009649,casort.org
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1001,NaN,NaN,2024-05-22 12:58:19.867931,2024-05-22 12:58:16.026061,2024-05-22 12:58:19.873527,NaN,2024-05-22 12:57:55.895852,ZDBFI-fuIDA.ZDBFI-fuIDA,"<font color=""#004f88""><b>TL;DR</b> Based on th...",NaN,"{""cardsV2"": [{""cardId"": ""aiResponseCard"", ""car...",121d41b5-e45c-4f43-b80b-bc7bfbd61c2b,can you prepare an employment tribunal schedul...,ZDBFI-fuIDA,NaN,nan
1009,NaN,NaN,2024-05-02 12:15:57.854832,2024-05-02 12:15:54.796790,2024-05-02 12:15:57.860820,NaN,2024-05-02 12:15:03.763731,GFE2wyMRLyk.Y8VVb1b-Qxg,"Based on the information provided, here is my ...",NaN,"{""cardsV2"": [{""cardId"": ""aiResponseCard"", ""car...",7ea184cd-42a9-475b-8969-6ee16a77adda,I am currently not working due to health issu...,GFE2wyMRLyk,NaN,nan
1020,NaN,NaN,2024-07-09 10:24:03.354152,2024-07-09 10:23:59.253884,2024-07-09 10:24:00.343873,NaN,2024-07-09 10:22:27.093322,rApqgiJRt5k.rApqgiJRt5k,"<font color=""#004f88""><b>Brief Summary:</b> <i...",NaN,{'cardsV2': [{'card': {'sections': [{'widgets'...,ce86b5d3-4d5f-4738-921d-00f8752e6e79,my neighbour wants to install pipes through m...,rApqgiJRt5k,NaN,nan
1033,NaN,NaN,NaN,2024-05-16 13:05:51.240441,NaN,NaN,2024-05-16 13:05:36.917624,SR17s_8y4yo.SR17s_8y4yo,"<font color=""#004f88""><b>TL;DR</b> Your client...",NaN,"{""cardsV2"": [{""cardId"": ""aiResponseCard"", ""car...",e3b57e02-0ccf-465d-8594-77aba62d68ac,Flexible working?,SR17s_8y4yo,NaN,nan


In [20]:
df_RCT = df_responses[(df_responses['Office'] == 'casort.org') | (df_responses['Office'] == 'citizensadvicemanchester.org.uk')]


In [22]:
df_RCT = pd.merge(left=df_RCT, right=df_evaluation, on='threadId', how='left')

In [25]:
df_RCT = df_RCT[(df_RCT['approvalTimestamp'] >= '2024-04-22') & (df_RCT['approvalTimestamp'] <= '2024-06-21')]

- Split out control and treatment groups
- Find approval rate
- Find survey responses

In [40]:
df_RCT_approval = df_RCT[['approved', 'approvalTimestamp']].groupby(by='approved').count()
df_RCT_approval['percentage'] = (df_RCT_approval['approvalTimestamp'] / df_RCT_approval['approvalTimestamp'].sum()) * 100
df_RCT_approval

excluded_users=[
    
'siobhan.durcan@citizensadvice.org.uk', 
'siobhan.durcan@casort.org.uk'
'AVarotsis@no10.gov.uk',
'kurtis.massey@casort.org',
'siobhan.durcan@casort.org',
'Chetan.chauhan-sims@ukhsa.gov.uk',
'hanna.harding@cabinetoffice.gov.uk',
'max.hollingdale@cabinetoffice.gov.uk',
'alex.moore@cabinetoffice.gov.uk'

]

,approvalTimestamp,percentage
approved,,
False,61,20.469799
True,237,79.530201


In [70]:
def rctGrouper(message):
    return 'control' if isinstance(message, (dict, list)) else 'treatment'       

df_RCT['Control or Treatment'] = df_RCT.apply(lambda x: rctGrouper(x['modulesUsed']), axis=1)

In [71]:
df_RCT.drop(df_RCT['approverEmail'] == )

,approvalTimestamp,approverEmail,approverReceivedTimestamp,llmResponseTimestamp,userThankedTimestamp,approved,llmPromptTimestamp,messageId,llmAnswer,supervisorMessage,...,Office,callComplete,surveyResponse,continueConversation,controlGroupMessage,moduleOutputs,callStart,modulesUsed,surveyCompleteTimestamp,Control or Treatment
2,2024-04-26 08:22:40.399397+00:00,ayesha.qudsia@casort.org,2024-04-26 08:20:43.892645,2024-04-26 08:20:40.564014,2024-04-26 08:20:43.898685,True,2024-04-26 08:19:35.942380,GbTFc_Eqd7U.GbTFc_Eqd7U,"Based on the information provided, you are gen...",None,...,casort.org,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,treatment
3,2024-05-24 11:12:40.044902+00:00,jamie.walker@citizensadvicemanchester.org.uk,2024-05-24 10:52:19.442129,2024-05-24 10:52:16.268586,2024-05-24 10:52:17.334049,True,2024-05-24 10:50:57.547456,_FJRGMey1mU._FJRGMey1mU,"<font color=""#004f88""><b>Brief Summary:</b> Fo...",None,...,citizensadvicemanchester.org.uk,True,[{'How confident did you feel in providing adv...,True,None,"{""randomisation"": [""continue_interaction"", ""tr...",24-05-2024 11:50,"{'module_arguments': {'split': '0.5', 'control...",24-05-2024 12:16:35,control
4,2024-05-21 15:38:44.571289+00:00,kurtis.massey@casort.org,2024-05-21 15:37:49.951078,2024-05-21 15:37:45.907813,2024-05-21 15:37:49.957946,True,2024-05-21 15:37:23.867283,NcGnst3Oaio.NcGnst3Oaio,"<font color=""#004f88""><b>TL;DR</b> Your client...",None,...,casort.org,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,treatment
6,2024-04-25 13:12:47.138140+00:00,jamie.mcglynn@casort.org,2024-04-25 13:12:12.333274,2024-04-25 13:12:09.570428,2024-04-25 13:12:12.340011,True,2024-04-25 13:11:23.807528,4BdewiS6-ww.ymMqb6qE1DY,"Based on the information provided, it seems yo...",None,...,casort.org,True,NaN,True,None,"{""randomisation"": [""continue_interaction"", ""tr...",NaN,"{'module_arguments': {'split': '0.5', 'control...",NaN,control
7,2024-05-31 08:41:21.197971+00:00,tom.hall@casort.org,2024-05-31 08:41:09.203845,2024-05-31 08:41:05.406203,2024-05-31 08:41:06.641394,False,2024-05-31 08:39:54.079991,CLLLZRrrryw.wvFbbwsKm84,"<font color=""#004f88""><b>Brief Summary:</b> Yo...",,...,casort.org,False,NaN,True,None,"{""randomisation"": [""continue_interaction"", ""tr...",31-05-2024 09:39,"{'module_arguments': {'split': '0.5', 'control...",NaN,control
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
454,2024-06-13 13:19:01.930026+00:00,andreas.varotsis@casort.org,2024-06-13 13:17:40.110556,2024-06-13 13:17:36.413772,2024-06-13 13:17:37.989006,True,2024-06-13 13:16:35.028559,Z0F7V3yo5Tg.5p7ZzX0Vci8,"<font color=""#004f88""><b>Brief Summary:</b> Yo...",None,...,casort.org,True,[{'How confident did you feel in providing adv...,True,None,"{""randomisation"": [""continue_interaction"", ""tr...",13-06-2024 14:16,"{'module_arguments': {'split': '0.5', 'control...",13-06-2024 14:19:46,control
455,2024-05-23 07:13:06.299359+00:00,kurtis.massey@casort.org,2024-05-23 07:13:02.001738,2024-05-23 07:12:58.577688,2024-05-23 07:12:59.889572,True,2024-05-23 07:11:58.565365,WYHCPe4txJ8.WYHCPe4txJ8,"<font color=""#004f88""><b>Brief Summary:</b> Yo...",None,...,casort.org,True,[{'How confident did you feel in providing adv...,True,None,"{""randomisation"": [""continue_interaction"", ""tr...",23-05-2024 08:11,"{'module_arguments': {'split': '0.5', 'control...",12-06-2024 14:55:14,control
456,2024-04-23 11:52:21.254776+00:00,mohammed.rahman@citizensadvicemanchester.org.uk,2024-04-23 11:44:50.801494,2024-04-23 11:44:47.106981,2024-04-23 11:44:50.810894,True,2024-04-23 11:44:06.954949,Ki31NbUp19g.Ki31NbUp19g,"Based on the information provided, here are th...",None,...,citizensadvicemanchester.org.uk,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,treatment
459,2024-05-17 12:05:09.223019+00:00,matthew.bayfield@citizensadvicemanchester.org.uk,2024-05-17 12:04:05.304798,2024-05-17 12:04:01.890038,2024-05-17 12:04:05.310609,True,2024-05-17 12:03:28.518127,aZeKRM8eo40.aZeKRM8eo40,"<font color=""#004f88""><b>